In [ ]:
# Inicialize o Otter
import otter
grader = otter.Notebook("lab09.ipynb")

# Laboratório 9: Regressão

Bem-vindo ao Laboratório 9!

Hoje vamos praticar regressão linear na prática. Você pode encontrar mais informações sobre este tópico no
[Capítulo 15.2](https://www.inferentialthinking.com/chapters/15/2/Regression_Line.html#the-regression-line).

**Submissão**: Uma vez que você tenha terminado, execute todas as células exceto a última, selecione Arquivo > Salvar Caderno, e então execute a célula final. Depois, submeta o arquivo zip baixado, que inclui seu caderno, de acordo com as instruções do seu instrutor.

In [ ]:
# Execute esta célula, mas por favor não a altere.

# Estas linhas importam os módulos Numpy e Datascience.
import numpy as np
from datascience import *

# Estas linhas fazem alguma mágica de plotagem sofisticada.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

import d8error

# 1. Quão Fiel é o Old Faithful?

Old Faithful é um gêiser no Parque Nacional de Yellowstone que é famoso por entrar em erupção em um cronograma relativamente regular. Execute a célula abaixo para ver o Old Faithful em ação!

In [ ]:
# Para os curiosos: esta é a forma de exibir um vídeo do YouTube em um
# caderno Jupyter. O argumento para YouTubeVideo é a parte
# da URL (chamada de "parâmetro de consulta") que identifica o
# vídeo. Por exemplo, a URL completa para este vídeo é:
#   https://www.youtube.com/watch?v=wE8NDuzt8eg
from IPython.display import YouTubeVideo
YouTubeVideo("wE8NDuzt8eg")

Algumas erupções do Old Faithful duram mais do que outras. Sempre que ocorre uma erupção longa, geralmente é seguida por uma espera ainda mais longa antes da próxima erupção. Se você visitar Yellowstone, talvez queira prever quando a próxima erupção acontecerá, para que possa ver o resto do parque em vez de esperar pelo gêiser.

Hoje, usaremos um conjunto de dados sobre durações de erupções e tempos de espera para ver se podemos fazer tais previsões com precisão usando regressão linear.

O conjunto de dados tem uma linha para cada erupção observada. Inclui as seguintes colunas:
- `duration`: Duração da erupção, em minutos
- `wait`: Tempo entre esta erupção e a próxima, também em minutos

Execute a próxima célula para carregar o conjunto de dados.

In [ ]:
faithful = Table.read_table("faithful.csv")
faithful

**Questão 1.0.** As seguintes afirmações são os passos desordenados da regressão linear.

1. Calcule os parâmetros da linha de regressão: a inclinação e a interseção.
2. Avalie a linha de regressão calculando o RMSE da linha e analisando o gráfico de resíduos.
3. Use a linha de regressão para gerar previsões para cada valor de x.
4. Determine se a regressão linear é um método razoável visualizando seus dados e calculando o coeficiente de correlação.

Faça um array chamado `least_squares_order` que contém a ordem correta de uma análise de regressão linear, onde o primeiro item do array é o primeiro passo de uma análise de regressão linear e o último item do array é o último passo de uma análise de regressão linear.

In [ ]:
least_squares_order = ...

In [ ]:
grader.check("q1_0")

Gostaríamos de usar a regressão linear para fazer previsões, mas isso não funcionará bem se os dados não estiverem aproximadamente relacionados de forma linear. Para verificar isso, devemos observar os dados.

**Questão 1.1.** Faça um gráfico de dispersão dos dados. É convencional colocar a coluna que queremos prever no eixo vertical e a outra coluna no eixo horizontal.

In [ ]:
...

**Questão 1.2.** A duração da erupção e o tempo de espera estão aproximadamente relacionados de forma linear com base no gráfico de dispersão acima? Essa relação é positiva?

_Digite sua resposta aqui, substituindo este texto._

Vamos continuar com a suposição de que eles estão relacionados de forma linear, então é razoável usar a regressão linear para analisar esses dados.

Em seguida, gostaríamos de plotar os dados em unidades padrão. Se você não se lembra da definição de unidades padrão, a seção do livro didático [14.2](https://www.inferentialthinking.com/chapters/14/2/Variability.html#standard-units) pode ajudar!

**Questão 1.3.** Calcule a média e o desvio padrão das durações das erupções e dos tempos de espera. **Em seguida**, crie uma tabela chamada `faithful_standard` contendo as durações das erupções e os tempos de espera em unidades padrão. As colunas devem ser nomeadas `duration (standard units)` e `wait (standard units)`.

In [ ]:
duration_mean = ...
duration_std = ...
wait_mean = ...
wait_std = ...

faithful_standard = Table().with_columns(
    "duration (standard units)", ...,
    "wait (standard units)", ...)
faithful_standard

In [ ]:
grader.check("q1_3")

**Questão 1.4.** Plote os dados novamente, mas desta vez em unidades padrão.

In [ ]:
...

Você notará que este gráfico parece o mesmo que o anterior! No entanto, os dados e os eixos estão escalados de forma diferente. Portanto, é importante ler as marcações nos eixos.

**Questão 1.5.** Entre os seguintes números, qual você acha que está mais próximo da correlação entre a duração da erupção e o tempo de espera neste conjunto de dados?

1. -1
2. 0
3. 1

Atribua `correlation` ao número correspondente ao seu palpite (1, 2 ou 3).

In [ ]:
correlation = ...

In [ ]:
grader.check("q1_5")

**Questão 1.6.** Calcule o coeficiente de correlação: `r`.

*Dica:* Use `faithful_standard`. A seção [15.1](https://www.inferentialthinking.com/chapters/15/1/Correlation.html#calculating-r) explica como fazer isso.

In [ ]:
r = ...
r

In [ ]:
grader.check("q1_6")

## 2. A linha de regressão
Lembre-se de que a **correlação** é a **inclinação da linha de regressão quando os dados são colocados em unidades padrão**.

A próxima célula plota a linha de regressão em unidades padrão:

$$\text{tempo de espera em unidades padrão} = r \times \text{duração da erupção em unidades padrão}$$

Em seguida, plota os dados em unidades padrão novamente, para comparação.

In [ ]:
def plot_data_and_line(dataset, x, y, point_0, point_1):
    """Faz um gráfico de dispersão do conjunto de dados, junto com uma linha passando por dois pontos."""
    dataset.scatter(x, y, label="data")
    xs, ys = zip(point_0, point_1)
    plots.plot(xs, ys, label="regression line")
    plots.legend(bbox_to_anchor=(1.5,.8))

plot_data_and_line(faithful_standard, 
                   "duration (standard units)", 
                   "wait (standard units)", 
                   [-2, -2*r], 
                   [2, 2*r])

Como você converteria um ponto em unidades padrão de volta para as unidades originais? Teríamos que "esticar" sua posição horizontal por `duration_std` e sua posição vertical por `wait_std`. Isso significa que o mesmo aconteceria com a inclinação da linha.

Esticar uma linha horizontalmente a torna menos íngreme, então dividimos a inclinação pelo fator de esticamento. Esticar uma linha verticalmente a torna mais íngreme, então multiplicamos a inclinação pelo fator de esticamento.

**Questão 2.1.** Calcule a inclinação da linha de regressão em unidades originais e atribua-a a `slope`.

(Se a explicação de "esticamento" não for intuitiva, consulte a seção [15.2](https://www.inferentialthinking.com/chapters/15/2/Regression_Line.html#the-equation-of-the-regression-line) no livro didático.)

In [ ]:
slope = ...
slope

In [ ]:
grader.check("q2_1")

Sabemos que a linha de regressão passa pelo ponto `(duration_mean, wait_mean)`. Lembre-se de que a equação da linha de regressão nas unidades originais é:

$$\text{tempo de espera} = \text{inclinação} \times \text{duração da erupção} + (- \text{inclinação} \times \text{duration\_mean} + \text{wait\_mean})$$

**Questão 2.2.** Calcule a interseção nas unidades originais e atribua-a a `intercept`. [Seção 15.2.5](https://inferentialthinking.com/chapters/15/2/Regression_Line.html#the-regression-line-in-standard-units) pode ser útil.

In [ ]:
intercept = ...
intercept

In [ ]:
grader.check("q2_2")

## 3. Investigando a linha de regressão
A inclinação e a interseção informam exatamente como a linha de regressão se parece. Para prever o tempo de espera para uma erupção, multiplique a duração da erupção por `slope` e depois some `intercept`.

**Questão 3.1.** Calcule o tempo de espera previsto para uma erupção que dura 2 minutos, e para uma erupção que dura 5 minutos.

In [ ]:
two_minute_predicted_waiting_time = ...
five_minute_predicted_waiting_time = ...

# Aqui está uma função auxiliar para imprimir suas previsões.
# Não modifique o código abaixo.
def print_prediction(duration, predicted_waiting_time):
    print("After an eruption lasting", duration,
          "minutes, we predict you'll wait", predicted_waiting_time,
          "minutes until the next eruption.")

print_prediction(2, two_minute_predicted_waiting_time)
print_prediction(5, five_minute_predicted_waiting_time)

In [ ]:
grader.check("q3_1")

A próxima célula plota a linha que passa entre esses dois pontos, que é (um segmento da) linha de regressão.

In [ ]:
plot_data_and_line(faithful, "duration", "wait", 
                   [2, two_minute_predicted_waiting_time], 
                   [5, five_minute_predicted_waiting_time])

**Questão 3.2.** Faça previsões para o tempo de espera após cada erupção na tabela `faithful`. (Claro, nós sabemos exatamente quais foram os tempos de espera! Estamos fazendo isso para podermos ver quão precisas são nossas previsões.) Coloque esses números em uma coluna em uma nova tabela chamada `faithful_predictions`. Sua primeira linha deve parecer com isso:

|duration|wait|predicted wait|
|-|-|-|
|3.6|79|72.1011|

*Dica:* Sua resposta pode ser apenas uma linha, embora você não esteja limitado a uma linha. Não há necessidade de um loop `for`; use aritmética de arrays em vez disso.

In [ ]:
faithful_predictions = ...
faithful_predictions

In [ ]:
grader.check("q3_2")

**Questão 3.3.** Quão próximos estávamos? Calcule o *resíduo* para cada erupção no conjunto de dados. O resíduo é o tempo de espera real menos o tempo de espera previsto. Adicione os resíduos a `faithful_predictions` como uma nova coluna chamada `residual` e nomeie a tabela resultante `faithful_residuals`.

*Dica:* Novamente, seu código será muito mais simples se você não usar um loop `for`.

In [ ]:
faithful_residuals = ...
faithful_residuals

In [ ]:
grader.check("q3_3")

Aqui está um gráfico dos resíduos que você calculou. Cada ponto corresponde a uma erupção. Ele mostra quanto nossa previsão superestimou ou subestimou o tempo de espera.

In [ ]:
faithful_residuals.scatter("duration", "residual", color="r")

Não há realmente um padrão nos resíduos, o que confirma que foi razoável tentar a regressão linear. É verdade que existem duas nuvens separadas; as durações das erupções pareciam cair em dois clusters distintos. Mas isso é apenas um padrão nas durações das erupções, não um padrão na relação entre as durações das erupções e os tempos de espera.

## 4. Quão precisas são as diferentes previsões?
Anteriormente, você deve ter descoberto que a correlação é bastante próxima de 1, então a linha se ajusta razoavelmente bem aos dados de treinamento. Isso significa que os resíduos são, em geral, pequenos (próximos de 0) em comparação com os tempos de espera.

Podemos ver isso visualmente ao plotar os tempos de espera e os resíduos juntos:

In [ ]:
# Apenas execute esta célula.
faithful_residuals.scatter("duration", "wait", label="actual waiting time", color="blue")
plots.scatter(faithful_residuals.column("duration"), faithful_residuals.column("residual"), label="residual", color="r")
plots.plot([2, 5], [two_minute_predicted_waiting_time, five_minute_predicted_waiting_time], label="regression line")
plots.legend(bbox_to_anchor=(1.7,.8));

No entanto, a menos que você tenha uma forte razão para acreditar que o modelo de regressão linear é verdadeiro, você deve ter cautela ao aplicar seu modelo de previsão a dados que são muito diferentes dos dados de treinamento.

**Questão 4.1.** Em `faithful`, nenhuma erupção durou exatamente 0, 2,5 ou 60 minutos. Usando essa linha, qual é o tempo de espera previsto para uma erupção que dura 0 minutos? 2,5 minutos? Uma hora?

In [ ]:
zero_minute_predicted_waiting_time = ...
two_point_five_minute_predicted_waiting_time = ...
hour_predicted_waiting_time = ...

print_prediction(0, zero_minute_predicted_waiting_time)
print_prediction(2.5, two_point_five_minute_predicted_waiting_time)
print_prediction(60, hour_predicted_waiting_time)

In [ ]:
grader.check("q4_1")

**Questão 4.2.** Para cada previsão, indique se você acha que é confiável e explique seu raciocínio.

_Digite sua resposta aqui, substituindo este texto._

## 5. Dividir e Conquistar

Parece, a partir do diagrama de dispersão, que existem dois grupos de pontos: um para durações em torno de 2 e outro para durações entre 3,5 e 5. Uma linha vertical em 3 divide os dois grupos.

In [ ]:
faithful.scatter("duration", "wait", label="actual waiting time", color="blue")
plots.plot([3, 3], [40, 100]);

A função `standardize` da aula aparece abaixo, que recebe uma tabela com colunas numéricas e retorna a mesma tabela com cada coluna convertida em unidades padrão.

In [ ]:
# Apenas execute esta célula.

def standard_units(any_numbers):
    """Converte qualquer array de números para unidades padrão."""
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)  

def standardize(t):
    """Retorna uma tabela na qual todas as colunas de t são convertidas para unidades padrão."""
    t_su = Table()
    for label in t.labels:
        t_su = t_su.with_column(label + ' (su)', standard_units(t.column(label)))
    return t_su

**Questão 5.1.** Calcule separadamente o coeficiente de correlação *r* para todos os pontos com duração abaixo de 3 **e depois** para todos os pontos com duração acima de 3. Para fazer isso, crie uma função que calcula `r` a partir de uma tabela, e então passe para ela duas tabelas diferentes de pontos, chamadas `below_3` e `above_3`.

*Dica:* Você pode assumir que a tabela não tem nenhum valor de duração que seja exatamente 3.

In [ ]:
def corr_coeff(t):
    """Retorna o coeficiente de regressão para as colunas 0 e 1."""
    t_su = standardize(t)
    ...

below_3 = ...
above_3 = ...
below_3_r = corr_coeff(below_3)
above_3_r = corr_coeff(above_3)
print("For points below 3, r is", below_3_r, "; for points above 3, r is", above_3_r)

In [ ]:
grader.check("q5_1")

**Questão 5.2.** Complete as funções `slope_of` e `intercept_of` abaixo.

Quando terminar, as funções `wait_below_3` e `wait_above_3` devem usar cada uma uma linha de regressão diferente para prever um tempo de espera para uma duração. A primeira função deve usar a linha de regressão para todos os pontos com duração abaixo de 3. A segunda função deve usar a linha de regressão para todos os pontos com duração acima de 3.

In [ ]:
def slope_of(table, r):
    """Retorna a inclinação da linha de regressão para a tabela em unidades originais.
    
    Assume que a coluna 0 contém os valores de x e a coluna 1 contém os valores de y.
    r é o coeficiente de regressão para x e y.
    """
    ...

def intercept_of(table, r):
    """Retorna o intercepto da linha de regressão para a tabela em unidades originais."""
    slope = slope_of(table, r)
    ...
    
below_3_slope = slope_of(below_3, below_3_r)
below_3_intercept = intercept_of(below_3, below_3_r)
above_3_slope = slope_of(above_3, above_3_r)
above_3_intercept = intercept_of(above_3, above_3_r)

def wait_below_3(duration):
    return below_3_slope * duration + below_3_intercept

def wait_above_3(duration):
    return above_3_slope * duration + above_3_intercept

In [ ]:
grader.check("q5_2")

O gráfico abaixo mostra as duas diferentes linhas de regressão, uma para cada grupo, junto com a linha de regressão original!

In [ ]:
faithful.scatter(0, 1)
plots.plot([2, 5], [two_minute_predicted_waiting_time, five_minute_predicted_waiting_time])
plots.plot([1, 3], [wait_below_3(1), wait_below_3(3)])
plots.plot([3, 6], [wait_above_3(3), wait_above_3(6)]);

**Questão 5.3.** Escreva uma função `predict_wait` que recebe uma `duration` (duração) e retorna o tempo de espera previsto usando a linha de regressão apropriada, dependendo se a duração é abaixo de 3 ou maior que (ou igual a) 3.

In [ ]:
def predict_wait(duration):
    ...

In [ ]:
grader.check("q5_3")

Os tempos de espera previstos para cada ponto aparecem abaixo.

In [ ]:
faithful_pred_split = faithful.with_column('predicted', faithful.apply(predict_wait, 'duration'))
faithful_pred_split.scatter(0)

<!-- BEGIN QUESTION -->

**Questão 5.4.** Você acha que as previsões produzidas por `predict_wait` seriam mais ou menos precisas do que as previsões da linha de regressão que você criou na seção 2? Como você poderia dizer?


_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

A célula a seguir irá plotar os resíduos para cada erupção no conjunto de dados quando temos uma linha de regressão e duas linhas de regressão. Também veremos a magnitude média dos valores residuais.

In [ ]:
# Apenas execute esta célula.
faithful_pred_split_residuals = faithful_pred_split.with_column('residual', faithful_pred_split.column(1) - faithful_pred_split.column(2))
plots.scatter(faithful_residuals.column('duration'), faithful_residuals.column('residual'), label='one regression line')
plots.scatter(faithful_pred_split_residuals.column('duration'), faithful_pred_split_residuals.column('residual'), label='two regression lines');
plots.axis([1, 6, -15, 15])
plots.legend(bbox_to_anchor=(1.5,.8));
print("Average Magnitude of Residual Values for One Regression Line: ", np.mean(abs(faithful_residuals.column('residual'))))
print("Average Magnitude of Residual Values for Two Regression Lines: ", np.mean(abs(faithful_pred_split_residuals.column('residual'))))

O gráfico de resíduos para os tempos de espera quando são previstos por duas linhas de regressão (vermelho) realmente não apresenta um padrão, o que confirma que também foi apropriado usar regressão linear no nosso cenário "Dividir para Conquistar". Como os dois gráficos de resíduos se comparam?

## 6. Submissão

<img src="lab09_pets.jpg" alt="desenho" width="500"/>

Parabéns, você concluiu o Laboratório 9!

**Passos importantes para a submissão:** 
1. Execute os testes e verifique se todos passaram.
2. Escolha **Salvar Caderno** no menu **Arquivo**, depois **execute a última célula**. 
3. Clique no link para baixar o arquivo zip.
4. Depois, submeta o arquivo zip para a tarefa correspondente de acordo com as instruções do seu instrutor. 

**É sua responsabilidade garantir que seu trabalho esteja salvo antes de executar a última célula.**

## Submissão

Certifique-se de ter executado todas as células no seu caderno em ordem antes de executar a célula abaixo, para que todas as imagens/gráficos apareçam na saída. A célula abaixo irá gerar um arquivo zip para você submeter. **Por favor, salve antes de exportar!**

In [ ]:
# Salve seu caderno primeiro, depois execute esta célula para exportar sua submissão.
grader.export(pdf=False, run_tests=True)